<a href="https://colab.research.google.com/github/1900690/Strawberry/blob/main/eye_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title 1.動画（.mp4）と視線データ(.json)をアップロード
#ライブラリをインポート
from pandas.core.base import doc
import shutil
import os
from google.colab import files
import pandas as pd
from PIL import Image
import cv2

アップロード方法を選択 = '\u30B0\u30FC\u30B0\u30EB\u30C9\u30E9\u30A4\u30D6\u304B\u3089\u30A2\u30C3\u30D7\u30ED\u30FC\u30C9' #@param ["グーグルドライブからアップロード", "ローカルからアップロード"]
#@markdown ※ドライブからアップロードする場合はファイル名を記載
ファイル名 = "itigo_230316152445" #@param {type:"string"}

if アップロード方法を選択 == 'ローカルからアップロード':
  #データをアップロード
  uploaded = files.upload()

  #ファイル名を取得
  ファイル名 = os.path.splitext(list(uploaded.keys())[0])[0]

if アップロード方法を選択 == 'グーグルドライブからアップロード':
  #グーグルドライブに接続
  from google.colab import drive
  drive.mount('/content/drive')

  #グーグルドライブからcontentにコピー
  shutil.copyfile("/content/drive/MyDrive/"+ファイル名+".mp4","/content/"+ファイル名+".mp4")
  shutil.copyfile("/content/drive/MyDrive/"+ファイル名+".json","/content/"+ファイル名+".json")

#視線データを読み込み必要な部分を抽出
json_file = "/content/"+ファイル名+".json"
df = pd.read_json(json_file)
df2=df["EyeData"].iloc[2]
df3 = pd.json_normalize(df2)
df6=df3[['recomValid','recomPoint.x','recomPoint.y']]

# 動画ファイルのキャプチャー
cap = cv2.VideoCapture("/content/"+ファイル名+".mp4")

# 動画ファイルのフレームレート取得
fps = cap.get(cv2.CAP_PROP_FPS)

#動画のサイズを取得
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width,height)

# 保存用動画ファイルのフォーマット設定
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('eye_camera.mp4', fourcc, fps, size)

# 動画を1コマずつ取り込んで処理
while(cap.isOpened()):
    ret, frame = cap.read() # キャプチャー画像の取り込み

    if ret==True: # キャプチャー画像がある場合
        # 現在時間を取得
        time = cap.get(cv2.CAP_PROP_POS_MSEC)

        # 現在のフレーム数を取得
        frame_n = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # タイムコードを作成
        timecode = str(int(time//1000)).zfill(2)+':'+str(int(frame_n%fps)).zfill(2)

        # 縦横半分にリサイズ
        #resize = cv2.resize(frame, size)

        # タイムコードを動画に追加
        cv2.drawMarker(frame, (int(df6.loc[int(frame_n)][1]*width),int(df6.loc[int(frame_n)][2]*height)), (0, 255, 255))

        # mp4動画を保存
        out.write(frame)

    else: # キャプチャー画像がない場合はループ終了
        break

cap.release() # 再生画像をクローズ
out.release() # 出力動画ファイルをクローズ

#ダウンロード
files.download("/content/eye_camera.mp4")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
